<b><font size=5><center>Исследование результатов А/В тестирования.<br> Решение тестового задания.</center></font></b>

## Описание проекта

Клиентам были отправлены два макета электронных писем (А и Б). Для каждого клиента известен
пол и возраст.
В приложенном файле результаты отработки кампании: доставлено писем, открыто писем,
сколько клиентов перешло по ссылке после открытия письма, сколько клиентов отписалось от
рассылки.
Необходимо оценить, как сегменты отреагировали на рассылку в рамках каждого макета? И
выяснить, какой из шаблонов в перспективных рассылках лучше всего отправлять клиентам?



##Шаг 1.Загрузка данных


In [24]:
#Подключим необходимые для работы библиотеки
import pandas as pd
from scipy import stats as st
import numpy as np
import math as mth

In [25]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [26]:
#Загрузим данные
df= pd.read_excel('/content/gdrive/MyDrive/Colab Notebooks/тестовое задание.xlsx')
df.head(10)

,Номер клиента,Макет,Доставлено,Открыто,Переход по ссылке,Отписки,Возраст,Пол
0,1,Шаблон Б,1.0,1,0,0,41.690411,М
1,2,Шаблон Б,1.0,1,0,0,28.249315,Ж
2,3,Шаблон Б,1.0,1,0,0,31.435616,М
3,4,Шаблон Б,1.0,1,0,0,34.564384,М
4,5,Шаблон Б,1.0,1,0,0,35.260274,М
5,6,Шаблон Б,1.0,1,0,1,26.476712,Ж
6,7,Шаблон Б,1.0,1,0,0,31.704110,М
7,8,Шаблон Б,1.0,1,0,1,33.471233,М
8,9,Шаблон Б,1.0,1,0,0,43.887671,М
9,10,Шаблон Б,1.0,1,0,0,36.649315,М


In [27]:
#Получим общую информацию о данных
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42126 entries, 0 to 42125
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Номер клиента      42126 non-null  int64  
 1   Макет              42126 non-null  object 
 2   Доставлено         23007 non-null  float64
 3   Открыто            42126 non-null  int64  
 4   Переход по ссылке  42126 non-null  int64  
 5   Отписки            42126 non-null  int64  
 6   Возраст            42126 non-null  float64
 7   Пол                42126 non-null  object 
dtypes: float64(2), int64(4), object(2)
memory usage: 2.6+ MB


In [28]:
#Получим информацию о пропусках
df.isna().sum()

Номер клиента            0
Макет                    0
Доставлено           19119
Открыто                  0
Переход по ссылке        0
Отписки                  0
Возраст                  0
Пол                      0
dtype: int64

In [29]:
#Проверм датасет на наличие яных дубликатов
df.duplicated().sum()

0

### Вывод
В результате исследования данных было устовновлено, что:
- в поле 'Доставлено' содержатся пропуски в значительном количестве. Ввиду того, что причину их появления не представляется возможным установить, примем решение обработать пропуски значением 0 и считать, что данное значение свидельствует о недоставленной клиенту рассылке;
- поле 'Возраст' представлено не цельночисленным типом данных (скорее всего, является результатом вычисления возраста на текущую дату). Предлагается заменить тип данных на целое число для группировки по возрасту (в группы будем относить по количеству полных лет, то есть 'откинем' значения после запятой, без математического округления);

Также для дальнейшей работы создадим поле с категорией возраста.


##Шаг 2. Предобработка данных

In [30]:
#Изменим тип данных в поле 'Возраст', обработаем пропуски и изменим тип данных в поле 'Доставлено'
df['Возраст'] = df['Возраст'].astype(int)
df['Доставлено'] = df['Доставлено'].fillna(0).astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42126 entries, 0 to 42125
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Номер клиента      42126 non-null  int64 
 1   Макет              42126 non-null  object
 2   Доставлено         42126 non-null  int64 
 3   Открыто            42126 non-null  int64 
 4   Переход по ссылке  42126 non-null  int64 
 5   Отписки            42126 non-null  int64 
 6   Возраст            42126 non-null  int64 
 7   Пол                42126 non-null  object
dtypes: int64(6), object(2)
memory usage: 2.6+ MB


In [31]:
#Напишем функцию категоризации возраста
def age_category(row):
    if row['Возраст'] <= 27:
        return 'до 27'
    if row['Возраст'] >= 41:
        return 'от 41'
    return 'от 28 до 40'
#Применим функцию к датасету и создадим новый столбец со значениями возрастной категории
df['Возраст_категория'] = df.apply(age_category, axis = 1)
df.head(10)

,Номер клиента,Макет,Доставлено,Открыто,Переход по ссылке,Отписки,Возраст,Пол,Возраст_категория
0,1,Шаблон Б,1,1,0,0,41,М,от 41
1,2,Шаблон Б,1,1,0,0,28,Ж,от 28 до 40
2,3,Шаблон Б,1,1,0,0,31,М,от 28 до 40
3,4,Шаблон Б,1,1,0,0,34,М,от 28 до 40
4,5,Шаблон Б,1,1,0,0,35,М,от 28 до 40
5,6,Шаблон Б,1,1,0,1,26,Ж,до 27
6,7,Шаблон Б,1,1,0,0,31,М,от 28 до 40
7,8,Шаблон Б,1,1,0,1,33,М,от 28 до 40
8,9,Шаблон Б,1,1,0,0,43,М,от 41
9,10,Шаблон Б,1,1,0,0,36,М,от 28 до 40


In [32]:
#Для удобства агрегирования данных создадим дополнительные поля
df['Клиенты'] = 1
df['Индекс'] = df['Возраст_категория']+df['Пол']
df.head(10)

,Номер клиента,Макет,Доставлено,Открыто,Переход по ссылке,Отписки,Возраст,Пол,Возраст_категория,Клиенты,Индекс
0,1,Шаблон Б,1,1,0,0,41,М,от 41,1,от 41М
1,2,Шаблон Б,1,1,0,0,28,Ж,от 28 до 40,1,от 28 до 40Ж
2,3,Шаблон Б,1,1,0,0,31,М,от 28 до 40,1,от 28 до 40М
3,4,Шаблон Б,1,1,0,0,34,М,от 28 до 40,1,от 28 до 40М
4,5,Шаблон Б,1,1,0,0,35,М,от 28 до 40,1,от 28 до 40М
5,6,Шаблон Б,1,1,0,1,26,Ж,до 27,1,до 27Ж
6,7,Шаблон Б,1,1,0,0,31,М,от 28 до 40,1,от 28 до 40М
7,8,Шаблон Б,1,1,0,1,33,М,от 28 до 40,1,от 28 до 40М
8,9,Шаблон Б,1,1,0,0,43,М,от 41,1,от 41М
9,10,Шаблон Б,1,1,0,0,36,М,от 28 до 40,1,от 28 до 40М


###Вывод
В результате предобработки данных:
 - изменены типы данных в полях 'Возраст', 'Доставлено';
 - обработаны пропуски в поле 'Доставлено';
 - клиенты были категоризированы в соответствии с возрастной группой (поле 'Возраст_категория');
 - добавлены поля для дальнейшей работы с данными.


##Шаг 3. Исследовательский анализ данных

In [33]:
#Представим датасет в виде сводной таблицы
pivot = pd.pivot_table(df,index = ['Макет','Возраст_категория', 'Пол'], values=['Клиенты','Доставлено', 'Открыто', 'Переход по ссылке', 'Отписки'], aggfunc='sum')
pivot = pivot[['Клиенты','Доставлено', 'Открыто', 'Переход по ссылке', 'Отписки']]
pivot

Клиенты  Доставлено  Открыто  \
Макет    Возраст_категория Пол                                 
Шаблон А до 27             Ж       1644         630       87   
                           М        577         229       32   
         от 28 до 40       Ж      10920        4236      761   
                           М       7585        2959      706   
         от 41             Ж       2878        1094      217   
                           М       2205         875      262   
Шаблон Б до 27             Ж       1075         865       63   
                           М        380         313       23   
         от 28 до 40       Ж       6956        5534      677   
                           М       4706        3721      632   
         от 41             Ж       1720        1393      219   
                           М       1480        1158      261   

                                Переход по ссылке  Отписки  
Макет    Возраст_категория Пол                              
Шаблон А до 27             Ж                   11        5  
                           М                    4        2  
         от 28 до 40       Ж                  155       72  
                           М                  122       69  
         от 41             Ж                   39       19  
                           М                   51       24  
Шаблон Б до 27             Ж                   12        8  
                           М                    4        5  
         от 28 до 40       Ж                  149       59  
                           М                  123       65  
         от 41             Ж                   50       14  
                           М                   52       16

#### Наблюдение 3.1
 **Судя по набору данных целевым действием клиентов является - переход по ссылке. Соответсвенно метрикой, на которую можно будет опираться при выборе макета, может быть отношение клиентов выполнивших целевое действие ко всем клиентам в рамках одного сегмента. Для расчета данной метрики добавили поле 'Клиенты' (=размерность сегмента). Также дополнительной метрикой может служить отношение отписок к клиентам в рамках сегмента. Отписки не являются целевым действием клиентов, вместе с тем метрика конверсий в отписчиков позволит оценить успешность шаблона (отписка = потерянный клиент).**

Дополнительно оценим конверсию в шаг на пути к целевому действию(Доставка-->Открытие-->Переход по ссылке)

*Также отмечено, что размерность сегментов для разных шаблонов разная (Количество клиентов в сегменте "Женщины до 27" для проведения теста было определено не равномерно: 1677 vs 1075).Данный факт свидельствует о неоптимальности разбиения данных по группам.
*

In [34]:
#Оценим метрики
df_metrics = pd.pivot_table(df,index = ['Макет','Возраст_категория', 'Пол'], values=['Клиенты','Переход по ссылке', 'Отписки'], aggfunc='sum')
df_metrics['Конв_целевое_действие,  %'] = round((df_metrics['Переход по ссылке']/df_metrics['Клиенты'])*100,2)
df_metrics['Конв_отписки,  %'] = round((df_metrics['Отписки']/df_metrics['Клиенты'])*100,2)
df_metrics = df_metrics[['Конв_целевое_действие,  %', 'Конв_отписки,  %']]
df_metrics

Конв_целевое_действие,  %  Конв_отписки,  %
Макет    Возраст_категория Пол                                             
Шаблон А до 27             Ж                         0.67              0.30
                           М                         0.69              0.35
         от 28 до 40       Ж                         1.42              0.66
                           М                         1.61              0.91
         от 41             Ж                         1.36              0.66
                           М                         2.31              1.09
Шаблон Б до 27             Ж                         1.12              0.74
                           М                         1.05              1.32
         от 28 до 40       Ж                         2.14              0.85
                           М                         2.61              1.38
         от 41             Ж                         2.91              0.81
                           М                         3.51              1.08

#### Наблюдение 3.2
**Значение метрик в рамках одного сегмента для разных шаблонов отличается. Вместе с тем, категорично судить о качественных отличиях в поведении клиентов по данным значениям нельзя, так как разница незначительна. Требуется проверить является ли эта разница в группах статистически значимой или нет. Для этого на следующем шаге проведем статистический анализ данных.**

In [35]:
#Оценим конверсию в шаг
cnv_step = pd.pivot_table(df,index = ['Макет','Возраст_категория', 'Пол'], values=['Клиенты','Доставлено', 'Открыто', 'Переход по ссылке'], aggfunc='sum')
cnv_step['Клиенты, %'] = round((cnv_step['Клиенты']/cnv_step['Клиенты'])*100,2)
cnv_step['Доставлено, %'] = round((cnv_step['Доставлено']/cnv_step['Клиенты'])*100,2)
cnv_step['Открыто, %'] = round((cnv_step['Открыто']/cnv_step['Доставлено'])*100,2)
cnv_step['Переход по ссылке, %'] = round((cnv_step['Переход по ссылке']/cnv_step['Открыто'])*100,2)
cnv_step =cnv_step[['Клиенты, %', 'Доставлено, %', 'Открыто, %', 'Переход по ссылке, %']]
cnv_step

Клиенты, %  Доставлено, %  Открыто, %  \
Макет    Возраст_категория Пол                                          
Шаблон А до 27             Ж         100.0          38.32       13.81   
                           М         100.0          39.69       13.97   
         от 28 до 40       Ж         100.0          38.79       17.97   
                           М         100.0          39.01       23.86   
         от 41             Ж         100.0          38.01       19.84   
                           М         100.0          39.68       29.94   
Шаблон Б до 27             Ж         100.0          80.47        7.28   
                           М         100.0          82.37        7.35   
         от 28 до 40       Ж         100.0          79.56       12.23   
                           М         100.0          79.07       16.98   
         от 41             Ж         100.0          80.99       15.72   
                           М         100.0          78.24       22.54   

                                Переход по ссылке, %  
Макет    Возраст_категория Пол                        
Шаблон А до 27             Ж                   12.64  
                           М                   12.50  
         от 28 до 40       Ж                   20.37  
                           М                   17.28  
         от 41             Ж                   17.97  
                           М                   19.47  
Шаблон Б до 27             Ж                   19.05  
                           М                   17.39  
         от 28 до 40       Ж                   22.01  
                           М                   19.46  
         от 41             Ж                   22.83  
                           М                   19.92

#### Наблюдение 3.3
**Данные о конверсии в шаг позволяют прийти к следующим выводам:**
- в среднем по 20 % клиентов переходило на следующий шаг после доставки рассылки в каждом сегменте
- скорее всего, существует какая-то техническая проблема, связанная с доставкой клиентам Шаблона А, значения во всех сегментах значительно хуже, чем в аналогичных сегментах Шаблона Б. Необходимо дополнительно исследовать данную проблему.

### Вывод
В результате исследовательского анализа данных было установлено:
 - тестовые группы в рамках 1 сегмента существенно отличаются по размерности, что в свою очередь может негативно влиять на результаты тестирования;
 - рассылка шаблона А практически в 2 раза хуже, чем рассылка шаблона Б, доставлялась клиентам во всех сегментах, возможной причиной являются проблемы технического характера;

 Для оценки поведения клиентов были выбраны две метрки:
 1. Конверсия целевого действия, которая показывает какая доля клиентов из сегмента достигла целевого действия.
 2. Конверсия отписчиков, которая показывает долю отписавшихся клиентов после получения рассылки.

Значения метрик в тестируемых группах каждого сегмента отличаются незначительно. Для принятия решения по выбору шаблона рассылки требуется проведение статистического анализа данных.


## Шаг 4. Статистический анализ данных


Для того, чтобы определить является ли разница между значениями метрик статистически значимой, проведем статистический анализ данных. Для оценки разницы долей применим z-тест для пропорций.

*Гипотезы для всех сегментов:*
- H0: доли клиентов, достигших целевое действие в тестовых группах, равны
- H1: доли клиентов, достигших целевое действие в тестовых группах, не равны

В качестве уровня статистической значимости возьмем 0,01. Такой уровень снижает вероятность получить ложнопозитивный результат стат. теста. Вместе с тем, если различие в выбоках действительно есть, то данный уровень не позволит сделать ее стат зачимой.

###Статистика 1
**Проверим, находят ли статистические критерии разницу между значениями метрики "Конверсия целевого действия" тестовых групп во всех сегментах (стат.значимость = 0,01)**

In [36]:
# Напишем функцию для проверки гипотезы о равенстве долей
def get_p_value(success1, success2, trials1, trials2):
    p1 = success1 / trials1  # пропорция успехов (конверсия) при отправке Макета А
    p2 = success2 / trials2 # пропорция успехов (конверсия) при отправке Макета Б
    p_combined = (success1 + success2) / (trials1 + trials2) # пропорция успехов в комбинированном датасете:
    difference = p1 - p2 # разница пропорций в датасетах
    z_value = difference / mth.sqrt(p_combined * (1 - p_combined) * (1/trials1 + 1/trials2)) # считаем статистику в ст.отклонениях стандартного нормального распределения
    distr = st.norm(0, 1) # задаем стандартное нормальное распределение (среднее 0, ст.отклонение 1)
    p_value = (1 - distr.cdf(abs(z_value))) * 2
    return p_value

In [37]:
# Подготовим набор данных с количеством клиентов, достигших целевое действие
sucsess = pd.pivot_table(df,index = 'Индекс', columns='Макет', values='Переход по ссылке', aggfunc='sum')
sucsess

Макет,Шаблон А,Шаблон Б
Индекс,,
до 27Ж,11,12
до 27М,4,4
от 28 до 40Ж,155,149
от 28 до 40М,122,123
от 41Ж,39,50
от 41М,51,52


In [38]:
# Подготовим набор данных с общим количеством клиентов
trials = pd.pivot_table(df,index = 'Индекс', columns='Макет', values='Клиенты', aggfunc='sum')
trials

Макет,Шаблон А,Шаблон Б
Индекс,,
до 27Ж,1644,1075
до 27М,577,380
от 28 до 40Ж,10920,6956
от 28 до 40М,7585,4706
от 41Ж,2878,1720
от 41М,2205,1480


In [39]:
# Проверим, находят ли статистические критерии разницу между тестовыми группами во всех сегментах
for x in [['Шаблон А', 'Шаблон Б']]:
    for i in sucsess.index:
        p_value = get_p_value(sucsess.loc[i,x[0]], sucsess.loc[i,x[1]], trials.loc[i,x[0]], trials.loc[i,x[1]])
        print('p-значение для:',i,x,p_value)
        if p_value < 0.01:
            print('Отвергаем нулевую гипотезу: между долями есть значимая разница')
        else:
            print('Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными')

p-значение для: до 27Ж ['Шаблон А', 'Шаблон Б'] 0.2131863013173536
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными
p-значение для: до 27М ['Шаблон А', 'Шаблон Б'] 0.5501860290876492
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными
p-значение для: от 28 до 40Ж ['Шаблон А', 'Шаблон Б'] 0.00026920891666515345
Отвергаем нулевую гипотезу: между долями есть значимая разница
p-значение для: от 28 до 40М ['Шаблон А', 'Шаблон Б'] 0.00010625820844656353
Отвергаем нулевую гипотезу: между долями есть значимая разница
p-значение для: от 41Ж ['Шаблон А', 'Шаблон Б'] 0.00021915446608478106
Отвергаем нулевую гипотезу: между долями есть значимая разница
p-значение для: от 41М ['Шаблон А', 'Шаблон Б'] 0.0301938934418593
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными


In [40]:
df_metrics

Конв_целевое_действие,  %  Конв_отписки,  %
Макет    Возраст_категория Пол                                             
Шаблон А до 27             Ж                         0.67              0.30
                           М                         0.69              0.35
         от 28 до 40       Ж                         1.42              0.66
                           М                         1.61              0.91
         от 41             Ж                         1.36              0.66
                           М                         2.31              1.09
Шаблон Б до 27             Ж                         1.12              0.74
                           М                         1.05              1.32
         от 28 до 40       Ж                         2.14              0.85
                           М                         2.61              1.38
         от 41             Ж                         2.91              0.81
                           М                         3.51              1.08

Статистические критерии нашли разницу в сегментах "от 28 до 40 Мужчины и Женщины", а также в сегменте "от 41 Женщины". По данным значений метрики "Конверсия целевого действия" Шаблон Б в указанных выше сегментах показал лучшую конверсию, чем Шаблнон А. В остальных сегментах разница оказалась статистически незначимой.

###Статистика 2
**Проверим, находят ли статистические критерии разницу между значениями метрики "Конверсия целевого действия" тестовых групп во всех сегментах (стат.значимость = 0,05)**


In [41]:
# Увеличим уровень стат. значимости и оценим результаты
for x in [['Шаблон А', 'Шаблон Б']]:
    for i in sucsess.index:
        p_value = get_p_value(sucsess.loc[i,x[0]], sucsess.loc[i,x[1]], trials.loc[i,x[0]], trials.loc[i,x[1]])
        print('p-значение для:',i,x,p_value)
        if p_value < 0.05:
            print('Отвергаем нулевую гипотезу: между долями есть значимая разница')
        else:
            print('Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными')

p-значение для: до 27Ж ['Шаблон А', 'Шаблон Б'] 0.2131863013173536
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными
p-значение для: до 27М ['Шаблон А', 'Шаблон Б'] 0.5501860290876492
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными
p-значение для: от 28 до 40Ж ['Шаблон А', 'Шаблон Б'] 0.00026920891666515345
Отвергаем нулевую гипотезу: между долями есть значимая разница
p-значение для: от 28 до 40М ['Шаблон А', 'Шаблон Б'] 0.00010625820844656353
Отвергаем нулевую гипотезу: между долями есть значимая разница
p-значение для: от 41Ж ['Шаблон А', 'Шаблон Б'] 0.00021915446608478106
Отвергаем нулевую гипотезу: между долями есть значимая разница
p-значение для: от 41М ['Шаблон А', 'Шаблон Б'] 0.0301938934418593
Отвергаем нулевую гипотезу: между долями есть значимая разница


Статистические критерии при увеличении уровня стат. значимости нашли разницу в сегменте "от 41 Мужчины". По данным значений метрик Шаблон Б также показал лучшую конверсию, чем Шаблон А.

###Статистика 3
**Проверим, находят ли статистические критерии разницу между значениями метрики "Конверсия отписчиков" тестовых групп во всех сегментах (стат.значимость = 0,01)**

*Гипотезы для всех сегментов:*
 - H0: доли отписок в тестовых группах равны
 - H1: доли отписок в тестовых группах не равны

In [42]:
# Подготовим набор данных по отпискам
sucsess_1 = pd.pivot_table(df,index = 'Индекс', columns='Макет', values='Отписки', aggfunc='sum')
sucsess_1

Макет,Шаблон А,Шаблон Б
Индекс,,
до 27Ж,5,8
до 27М,2,5
от 28 до 40Ж,72,59
от 28 до 40М,69,65
от 41Ж,19,14
от 41М,24,16


In [43]:
# Проверим, находят ли статистические критерии разницу между тестовыми группами во всех сегментах
for x in [['Шаблон А', 'Шаблон Б']]:
    for i in sucsess.index:
        p_value = get_p_value(sucsess_1.loc[i,x[0]], sucsess_1.loc[i,x[1]], trials.loc[i,x[0]], trials.loc[i,x[1]])
        print('p-значение для:',i,x,p_value)
        if p_value < 0.01:
            print('Отвергаем нулевую гипотезу: между долями есть значимая разница')
        else:
            print('Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными')

p-значение для: до 27Ж ['Шаблон А', 'Шаблон Б'] 0.10386568466030366
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными
p-значение для: до 27М ['Шаблон А', 'Шаблон Б'] 0.08514766456761902
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными
p-значение для: от 28 до 40Ж ['Шаблон А', 'Шаблон Б'] 0.1489282097948852
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными
p-значение для: от 28 до 40М ['Шаблон А', 'Шаблон Б'] 0.014404458154688049
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными
p-значение для: от 41Ж ['Шаблон А', 'Шаблон Б'] 0.5500281594316667
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными
p-значение для: от 41М ['Шаблон А', 'Шаблон Б'] 0.9831490711885882
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными


Отписывались клиенты одинаково во всех тестовых группах во всех сегментах

###Статистика 4
**Проверим, находят ли статистические критерии разницу между значениями метрики "Конверсия отписчиков" тестовых групп во всех сегментах (стат.значимость = 0,05)**

In [44]:
#Также поднимем уровень стат значимости
for x in [['Шаблон А', 'Шаблон Б']]:
    for i in sucsess.index:
        p_value = get_p_value(sucsess_1.loc[i,x[0]], sucsess_1.loc[i,x[1]], trials.loc[i,x[0]], trials.loc[i,x[1]])
        print('p-значение для:',i,x,p_value)
        if p_value < 0.05:
            print('Отвергаем нулевую гипотезу: между долями есть значимая разница')
        else:
            print('Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными')

p-значение для: до 27Ж ['Шаблон А', 'Шаблон Б'] 0.10386568466030366
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными
p-значение для: до 27М ['Шаблон А', 'Шаблон Б'] 0.08514766456761902
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными
p-значение для: от 28 до 40Ж ['Шаблон А', 'Шаблон Б'] 0.1489282097948852
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными
p-значение для: от 28 до 40М ['Шаблон А', 'Шаблон Б'] 0.014404458154688049
Отвергаем нулевую гипотезу: между долями есть значимая разница
p-значение для: от 41Ж ['Шаблон А', 'Шаблон Б'] 0.5500281594316667
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными
p-значение для: от 41М ['Шаблон А', 'Шаблон Б'] 0.9831490711885882
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными


Статистические критерии при увеличении уровня стат значимости уловили разницу значений метрики в сегменте "от 28 до 40 Мужчины", согласно значению метрики "Конверсия отписчиков" клиенты данного сегмента чаще (на 0,4 %) отписывались от рассылки с макетом Б, чем от макета с Шаблоном А.

## Общий вывод и рекомендации

В результате проведенного исследования в качестве критериев оценки поведения клиентов в целях выбора переспективного макета рассылки были выбраны две метрики:
 -  конверсия целевого действия, которая показывает какая доля клиентов из сегмента достигла целевого действия (перешли по ссылке).
 - конверсия отписчиков, которая показывает долю отписавшихся клиентов после получения рассылки.

Полученные значения метрик в тестовых группах во всех сегментах оказались с незначительной разницей, по которой трудно однозначно сделать выбор в пользу того или иного макета рассылки.

Для того, чтобы определить является ли разница в группах статистически значимой, использовали z-тест равенства долей.

Результаты статистического анализа показали, что шаблон Б в сегментах:
 - "от 28 до 40 Мужчины"
 - "от 28 до 40 Женщины"
 - "от 41 Мужчины"
 - "от 41 Женщины"

 показал лучшую конверсию целевого действия, чем шаблон А.

В остальных сегментах разница значений метрики "конверсии целевого действия" в тестовых группах оказалась статистически не значимой.

Также статистический критерий выявил разницу между значениями метрики "Конверсия отписчиков" в сегменте "от 28 до 40 Мужчины". Клиненты данного сегмена на 0,4 % чаще отписывались от рассылки шаблона Б, чем клиенты, получившие шаблон А. В остальных сегментах клиенты одинаково отписывались от рассылки (Шаблон А и Шаблон Б).

Таким образом, на основании полученных результатов можно прийти к выводу, что наиболее переспективным макетом для рассылки может быть шаблон Б. Для 4-х сегментов из 6 он показал лучшую конверсию, чем шаблон А, для остальных 2 конверсия одинаковая. Также несмотря на то, что в сегменте "от 28 до 40 Мужчины" клиенты отписывались от шаблона Б чаще, целевых действий шаблон принес на 1% больше, чем шаблон А.

